In [3]:
!python -m spacy download en_core_web_lg

     ------------------------------------- 587.7/587.7 MB 11.9 MB/s eta 0:00:00

D:\anaconda\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(



✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [8]:
import json
cv_data = json. load(open('dataset.json','r'))

In [9]:
len(cv_data)

1014

In [10]:
def get_spacy_doc(file, data):
    nlp = spacy.blank("en")
    db = DocBin()

    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot ['entities']

        ents = []
        entity_indices = []

        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity == True:
                continue

            entity_indices = entity_indices + list(range(start,end))

            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except:
                continue

            if span is None:
                err_data = str([start,end]) + "  "+ str(text) + "\n"
                file.write(err_data)

            else:
                ents.append(span)


        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass

    return db

In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [12]:
len(train), len(test)

(709, 305)

In [16]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
file = open('train_file.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data,spacy' )

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

100%|███████████████████████████████████████████████████████████████████████████████| 305/305 [00:02<00:00, 111.62it/s]


In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./train_data.spacy --paths.dev ./test_data.spacy